# Taller 2 - Exploración de ChromaDB como Base de Datos Vectorial

**Estudiante:** John Alexander Chicaiza  
**Programa:** Especialización en Construcción de Software  
**Universidad:** Universidad de Nariño  
**Docente:** PhD. Oscar Ceballos  
**Curso:** Bases de Datos Vectoriales  
**Fecha:** Octubre 2025

## Objetivos del Taller

Este taller explora **ChromaDB** como alternativa a Pinecone, implementando todas las operaciones fundamentales de una base de datos vectorial:

-  **Instalación y configuración** del sistema
-  **Creación de colecciones** (equivalente a índices)
-  **Inserción de vectores** con metadatos
-  **Consultas de similitud** vectorial
-  **Actualización** de vectores existentes
-  **Eliminación** de vectores
-  **Análisis comparativo** con otros motores vectoriales

---

## Instalación y Configuración de ChromaDB

### Instalación con Docker Compose (Configuración del Proyecto)
```bash
# Iniciar ChromaDB con docker-compose
docker-compose up -d

# Verificar que el servicio esté corriendo
docker-compose ps
```

El archivo `docker-compose.yml` incluido configura:
- **Imagen:** `ghcr.io/chroma-core/chroma:latest`
- **Puerto:** 8000 (mapeado a localhost)
- **Persistencia:** Volumen `./chroma_data` montado
- **CORS:** Habilitado para desarrollo
- **Reset:** Permitido para pruebas

### Configuración del Cliente Python
- **Modo:** Cliente HTTP (servidor independiente)
- **Host:** localhost
- **Puerto:** 8000
- **Persistencia:** Datos almacenados en `chroma_data/`

### Configuración del Entorno Virtual Python

Para el desarrollo del taller se configuró un entorno virtual de Python con el fin de aislar las dependencias del proyecto y garantizar un entorno limpio y reproducible. 

**Pasos de configuración realizados:**

```bash
# 1. Crear el entorno virtual
python3 -m venv chroma_env

# 2. Activar el entorno virtual
source chroma_env/bin/activate

# 3. Actualizar el gestor de paquetes
pip install --upgrade pip

# 4. Instalar dependencias necesarias
pip install jupyter chromadb

# 5. Verificar instalación 
jupyter notebook  # O abrir directamente en VS Code
```

**Entornos de desarrollo utilizados:**
- **Visual Studio Code**: Editor principal con extensión de Jupyter para notebooks
- **Jupyter Notebook**: Entorno web alternativo para desarrollo interactivo
- **Terminal integrada**: Para comandos Docker y gestión del entorno virtual

**Estructura del entorno creado:**
- `chroma_env/` - Directorio del entorno virtual
- `chroma_env/bin/` - Ejecutables (Python, pip, jupyter)
- `chroma_env/lib/` - Librerías instaladas (chromadb, jupyter, etc.)

Este setup permite desarrollar y documentar paso a paso las operaciones CRUD y las consultas semánticas sobre ChromaDB desde un entorno local y controlado, utilizando tanto VS Code como Jupyter según la preferencia del momento.

---

## 1. Conexión al Servidor ChromaDB

**Explicación técnica:** ChromaDB puede ejecutarse en modo embebido o como servidor HTTP. Aquí usamos el modo servidor para simular un entorno de producción.

In [24]:
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    settings=Settings(allow_reset=True)
)
print("✅ Conectado a Chroma en http://localhost:8000")

✅ Conectado a Chroma en http://localhost:8000


In [25]:
collection = client.create_collection(name="demo_vectores", get_or_create=True)
print("✅ Colección disponible:", collection.name)

✅ Colección disponible: demo_vectores


## 2. Creación de Colección

**Concepto:** En ChromaDB, una **colección** es equivalente a un **índice** en Pinecone. Agrupa vectores relacionados y define la función de distancia a utilizar.

In [26]:
collection.add(
    ids=["v1", "v2", "v3"],
    embeddings=[
        [0.1, 0.2, 0.3],
        [0.4, 0.5, 0.6],
        [0.9, 0.8, 0.7]
    ],
    metadatas=[
        {"nombre": "vector 1"},
        {"nombre": "vector 2"},
        {"nombre": "vector 3"}
    ]
)
print("✅ Vectores insertados correctamente.")

✅ Vectores insertados correctamente.


##  3. Inserción de Vectores (CREATE)

**Operación fundamental:** Insertamos vectores de 3 dimensiones con sus metadatos asociados. ChromaDB automáticamente calcula y almacena las representaciones optimizadas para búsqueda.

In [27]:
result = collection.query(
    query_embeddings=[[0.1, 0.2, 0.25]],
    n_results=2,
    include=["distances", "metadatas"]
)
print("🔎 Resultados de la consulta:")
print(result)

🔎 Resultados de la consulta:
{'ids': [['v1', 'v2']], 'distances': [[0.002500001, 0.3025]], 'embeddings': None, 'metadatas': [[{'nombre': 'vector 1'}, {'nombre': 'vector 2'}]], 'documents': None, 'uris': None, 'data': None, 'included': ['distances', 'metadatas']}


## 4. Consulta de Similitud (READ)

**Búsqueda vectorial:** Utilizamos un vector de consulta `[0.1, 0.2, 0.25]` para encontrar los 2 vectores más similares. ChromaDB usa **distancia coseno** por defecto.

In [28]:
collection.update(
    ids=["v1"],
    embeddings=[[0.2, 0.2, 0.2]],
    metadatas=[{"nombre": "vector 1 (actualizado)"}]
)
print("✅ Vector v1 actualizado.")

✅ Vector v1 actualizado.


##  5. Actualización de Vectores (UPDATE)

**Modificación en sitio:** ChromaDB permite actualizar tanto el embedding como los metadatos de vectores existentes usando su ID único.

In [29]:
collection.delete(ids=["v3"])
print("🗑️ Vector v3 eliminado.")

🗑️ Vector v3 eliminado.


##  6. Eliminación de Vectores (DELETE)

**Limpieza de datos:** Eliminamos vectores específicos por ID. Esta operación es irreversible y libera espacio en el índice.

In [30]:
print("Colecciones actuales:", [c.name for c in client.list_collections()])

Colecciones actuales: ['demo_vectores']


In [31]:
info = collection.get()
print("📘 Información de la colección:")
print(info)

📘 Información de la colección:
{'ids': ['v1', 'v2'], 'embeddings': None, 'metadatas': [{'nombre': 'vector 1 (actualizado)'}, {'nombre': 'vector 2'}], 'documents': [None, None], 'data': None, 'uris': None, 'included': ['metadatas', 'documents']}


## Análisis Comparativo: ChromaDB vs Otras Bases de Datos Vectoriales

### ChromaDB vs Pinecone vs Milvus vs Weaviate vs FAISS

| Característica | ChromaDB | Pinecone | Milvus | Weaviate | FAISS |
|----------------|----------|----------|---------|----------|-------|
| **Tipo** | Open Source | SaaS | Open Source | Open Source | Biblioteca |
| **Instalación** |  Simple |  Solo nube |  Compleja | Media |  Pip install |
| **Escalabilidad** |  Media |  Alta |  Alta |  Alta |  Local |
| **Costo** |  Gratis |  Pago |  Gratis |  Gratis |  Gratis |
| **Metadatos** |  Nativo |  Nativo |  Nativo |  Avanzado |  Manual |
| **APIs** | Python, REST | Python, REST | Python, Go, Java | Python, GraphQL | Python |
| **Persistencia** |  SQLite/DuckDB |  Nativa |  Configurable |  Múltiples |  RAM |

### Ventajas Específicas de ChromaDB

1. **Simplicidad de Uso**: API intuitiva y instalación directa
2. **Desarrollo Local**: Perfecto para prototipado y desarrollo
3. **Múltiples Modos**: Embebido, cliente-servidor, persistente
4. **Sin Vendor Lock-in**: Open source completo
5. **Integración Python**: Diseñado específicamente para el ecosistema Python/ML

### Limitaciones Identificadas

1. **Escalabilidad**: Menos optimizado para millones de vectores
2. **Algoritmos**: Menos opciones de índices que Milvus
3. **Ecosistema**: Menor integración con herramientas empresariales
4. **Consultas Complejas**: Menos expresivo que Weaviate con GraphQL

---

## Conclusiones del Taller

###  Objetivos Cumplidos

-  **Instalación exitosa** de ChromaDB en modo servidor
-  **Configuración completa** del cliente HTTP
-  **Implementación de CRUD** completo para vectores
-  **Verificación de operaciones** con evidencias
-  **Análisis comparativo** con otras soluciones

###  Observaciones Técnicas

1. **ChromaDB es ideal para**:
   - Proyectos de investigación y prototipado
   - Aplicaciones Python-first
   - Desarrollos que requieren control total del stack

2. **Diferencias conceptuales clave**:
   - **Colecciones** vs **Índices** (terminología)
   - **Distancia coseno** por defecto vs configuración explícita
   - **Metadatos integrados** vs sistemas separados

###  Caso de Uso Recomendado

ChromaDB es perfecto para **MVPs**, **investigación académica** y **aplicaciones medianas** donde se valora más la simplicidad y control que la escala extrema.

---

**  Archivos del Entregable:**
- `ChromaDB_OperacionesCRUD_Vectoriales.ipynb` - Notebook con implementación completa
- `docker-compose.yml` - Configuración de infraestructura  
- `chroma_data/` - Datos persistentes de ChromaDB